In [ ]:
def merge_sims_to_batch(self):
        Simulation_path = "/mnt/analysis/e17023/Adam/simulations/"
        Queue_path = "/mnt/analysis/e17023/Adam/gadget_analysis/Simulations/Output/"
        
        batch_number = self.sim_batch_number_entry.get()
        batch_name = f"run_s{batch_number}"
        
        print(f"Merging simulations to batch {batch_name}")
        
        # check if the batch already exists
        if not os.path.exists(f"{Simulation_path}{batch_name}"):
            os.makedirs(f"{Simulation_path}{batch_name}")
            os.makedirs(f"{Simulation_path}{batch_name}/component_h5")
        
        subprocess.run(f"cp {Simulation_path}parameters.csv {Simulation_path}{batch_name}/component_h5/param-{time.time()}.csv", shell=True, check=True)
        
        # move all component files to batch
        sim_files = os.listdir(Queue_path)
        for sim_file in sim_files:
            if sim_file.endswith(".h5"):
                os.rename(f"{Queue_path}{sim_file}", f"{Simulation_path}{batch_name}/component_h5/{sim_file}")
        
        # create the batch h5 file
        if os.path.exists(f"{Simulation_path}{batch_name}.h5"):
            os.remove(f"{Simulation_path}{batch_name}.h5")
        
        if len(os.listdir(f"{Simulation_path}{batch_name}/component_h5")) < 1:
            print("No component files found for batch.")
            return
        
        component_h5_list = [filename for filename in os.listdir(f"{Simulation_path}{batch_name}/component_h5") if filename.endswith('.h5')]
        component_h5_list.sort()
        
        # copy first file as template
        template_h5 = component_h5_list.pop(0)
        os.system(f"cp {Simulation_path}{batch_name}/component_h5/{template_h5} {Simulation_path}{batch_name}.h5")
        
        for component_h5 in component_h5_list:
            batch_h5 = h5py.File(f"{Simulation_path}{batch_name}.h5", 'a')
            component_h5 = h5py.File(f"{Simulation_path}{batch_name}/component_h5/{component_h5}", 'r')
            
            # get the event number of the last event in the batch
            last_event_number = int(batch_h5['meta/meta'][2]) + 1
            
            # merge get data
            for key in component_h5['get'].keys():
                if 'data' in key:
                    try:
                        event_number = int(key.split('_')[0][3:])
                        batch_h5.create_dataset(f"get/evt{event_number + last_event_number}_header", data=component_h5[f"get/{key}"], dtype='float64')
                        batch_h5.create_dataset(f"get/evt{event_number + last_event_number}_data", data=component_h5[f"get/{key}"], dtype='int16')
                        batch_h5[f'get/evt{event_number + last_event_number}_header'][0] = event_number + last_event_number # update the event number in the header
                    except IndexError:
                        print(f"Error in {component_h5} - {key}")
            meta_data = batch_h5['meta/meta'] 
            
            # merge clouds data
            for key in component_h5['clouds'].keys():
                event_number = int(key.split('_')[0].split('t')[1]) # evt#_cloud
                batch_h5.create_dataset(f"clouds/evt{event_number + last_event_number}_cloud", data=component_h5[f"clouds/{key}"], dtype='float64')
                meta_data[2] = max(meta_data[2], event_number + last_event_number)
            
            # re-write the meta data
            batch_h5['meta'].pop('meta')
            batch_h5.create_dataset('meta/meta', data=meta_data, dtype='float64')
            
            # close the files
            component_h5.close()
            batch_h5.close()
